In [30]:
import numpy as np
import os
import json
from random import shuffle
import tensorflow as tf
import tensorflow.keras as keras
from sklearn.model_selection import train_test_split


In [48]:
# Load all the data from the json files
dir = r"src/dataFiveNotePatterns/"
positive_note_patterns = []
negative_note_patterns = []

for file in os.listdir(dir):
    with open(dir + file, "r", encoding='utf-8') as f:
        data = json.load(f)
        positive_note_patterns = data["five_notes_patterns"]
        negative_note_patterns = data["five_notes_patterns"]


print("Positive note patterns: ", len(positive_note_patterns))
print("Negative note patterns: ", len(negative_note_patterns))


Positive note patterns:  170
Negative note patterns:  170


In [49]:
## make sure the two lists have the same length
if(len(positive_note_patterns) > len(negative_note_patterns)):
    positive_note_patterns = positive_note_patterns[:len(negative_note_patterns)]
else:
    negative_note_patterns = negative_note_patterns[:len(positive_note_patterns)]


all_note_patterns = positive_note_patterns + negative_note_patterns


# shuffle the data
shuffle(all_note_patterns)


In [50]:
all_fret_patterns = []
all_fret_patterns_labels = []
for pattern in all_note_patterns:
    all_fret_patterns.append([fret for _, fret, _ in pattern[:-1]])
    all_fret_patterns_labels.append(pattern[-1])


all_fret_patterns = np.array(all_fret_patterns)
all_fret_patterns_labels = np.array(all_fret_patterns_labels)

# split the data into training and testing
train_ratio = 0.8
X_train, X_test, y_train, y_test = train_test_split(all_fret_patterns, all_fret_patterns_labels, test_size=1-train_ratio)
for x, y in zip(X_train, y_train):
    print(x, y)

[3 5 1 3] [     5      3 563760]
[0 2 2 1] [     5      0 270720]
[0 2 3 2] [     1      2 111840]
[5 8 2 3] [    3     2 23040]
[3 2 0 1] [     3      2 335520]
[0 2 3 2] [     3      0 282240]
[5 8 5 7] [     3      5 465600]
[7 5 7 5] [     4      7 468240]
[17 20 17 17] [     1     20 526080]
[13 14 13 15] [     2     13 486240]
[10 12 10 12] [     4     14 475440]
[3 2 0 1] [     3      2 400800]
[14 12 14 14] [     3     14 512640]
[5 7 5 7] [     4      5 472560]
[2 1 0 1] [    3     2 12000]
[13 14 13 15] [     2     13 487200]
[3 2 0 1] [     3      2 270240]
[7 5 7 5] [     4      7 468240]
[5 5 5 7] [   2    5 3360]
[1 2 3 0] [    5     0 99840]
[5 7 5 3] [     6      5 565440]
[ 8 12 10 10] [     4     12 474240]
[1 2 0 0] [     2      0 415200]
[0 1 2 0] [     4      2 190560]
[3 2 0 1] [     5      2 325440]
[7 7 6 8] [    2     5 66720]
[0 2 0 0] [     4      0 105600]
[0 0 7 5] [     1      5 140640]
[20 20 19 19] [     3     17 502720]
[4 3 2 1] [     1      0 147600]


In [51]:
#build the model
model = keras.Sequential([
    keras.layers.Dense(64, input_shape=(X_train.shape[1],), activation='tanh'),
    keras.layers.Dense(64, activation='tanh'),
    keras.layers.Dense(64, activation='tanh'),
    keras.layers.Dense(3, activation='sigmoid')
    #keras.layers.Dense(3, activation='softmax')
])

print("Model Summary", model.summary())

Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_35 (Dense)                │ (None, 64)             │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_36 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_37 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_38 (Dense)                │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,835 (34.51 KB)

 Trainable params: 8,835 (34.51 KB)

 Non-trainable params: 0 (0.00 B)

Model Summary None


In [52]:
print(X_train)
print(y_train)
model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3), loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=['accuracy'])
model.fit(X_train, y_train, epochs=200, batch_size=64)

[[3 5 1 3]
 [0 2 2 1]
 [0 2 3 2]
 ...
 [3 2 2 3]
 [5 3 3 1]
 [2 5 4 0]]
[[     5      3 563760]
 [     5      0 270720]
 [     1      2 111840]
 [     3      2  23040]
 [     3      2 335520]
 [     3      0 282240]
 [     3      5 465600]
 [     4      7 468240]
 [     1     20 526080]
 [     2     13 486240]
 [     4     14 475440]
 [     3      2 400800]
 [     3     14 512640]
 [     4      5 472560]
 [     3      2  12000]
 [     2     13 487200]
 [     3      2 270240]
 [     4      7 468240]
 [     2      5   3360]
 [     5      0  99840]
 [     6      5 565440]
 [     4     12 474240]
 [     2      0 415200]
 [     4      2 190560]
 [     5      2 325440]
 [     2      5  66720]
 [     4      0 105600]
 [     1      5 140640]
 [     3     17 502720]
 [     1      0 147600]
 [     2     13 494880]
 [     3      5 453120]
 [     1      2  95040]
 [     3      0 347520]
 [     3      5 124320]
 [     2     15 477120]
 [     2     13 489120]
 [     5      5 571680]
 [     2     15 